In [39]:
from Ichigaya import chart
刹那旅行 = chart.chart(296, "special")
刹那旅行.to_path("谱面")
json = 刹那旅行.load()

In [65]:
chart_keys = {
    "Single": [], 
    "Hold": [], 
    "Flick": [], 
    "Hold&Flick": [], 
    "Left": [], 
    "Right": []
}
time_stamps = []
for i in json:
    if i["type"] == "BPM":
        time_stamps.append(i)
    elif i["type"] == "Single":
        if "flick" in i.keys():
            chart_keys["Flick"].append(i)
        else:
            chart_keys["Single"].append(i)
    elif i["type"] == "Directional":
        if i["direction"] == "Left":
            chart_keys["Left"].append(i)
        else:
            chart_keys["Right"].append(i)
    elif i["type"] != "System":
        if "flick" in i["connections"][-1].keys():
            chart_keys["Hold&Flick"].append(i)
        else:
            chart_keys["Hold"].append(i)


In [66]:
time_stamps

[{'type': 'BPM', 'bpm': 73, 'beat': 0},
 {'type': 'BPM', 'bpm': 72, 'beat': 9},
 {'type': 'BPM', 'bpm': 68, 'beat': 10},
 {'type': 'BPM', 'bpm': 74, 'beat': 11},
 {'type': 'BPM', 'bpm': 72, 'beat': 12},
 {'type': 'BPM', 'bpm': 74, 'beat': 14},
 {'type': 'BPM', 'bpm': 72, 'beat': 15},
 {'type': 'BPM', 'bpm': 76, 'beat': 16},
 {'type': 'BPM', 'bpm': 117, 'beat': 20},
 {'type': 'BPM', 'bpm': 145, 'beat': 22}]

In [67]:
shadow = {}
for single in chart_keys["Single"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

In [68]:
for hold in chart_keys["Hold"]:
    for node in [hold["connections"][0], hold["connections"][-1]]:
        beat = str(node["beat"])
        if beat in shadow.keys():
            shadow[beat].append(node["lane"])
        else:
            shadow.update({beat: [node["lane"]]})

In [69]:
for hold in chart_keys["Hold&Flick"]:
    for node in [hold["connections"][0], hold["connections"][-1]]:
        beat = str(node["beat"])
        if beat in shadow.keys():
            shadow[beat].append(node["lane"])
        else:
            shadow.update({beat: [node["lane"]]})

In [70]:
for single in chart_keys["Flick"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

for single in chart_keys["Left"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

for single in chart_keys["Right"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

In [71]:
links = {}
for stamp in shadow.keys():
    if len(shadow[stamp]) > 1:
        links.update({stamp: shadow[stamp]})
print(links)

{'5': [1, 4], '8': [6, 3], '9': [1, 4], '10': [5, 2], '15.5': [3, 1], '23': [3, 6], '24.5': [1, 5], '26.25': [6, 2], '27.5': [1, 4], '28': [5, 0], '28.5': [6, 0], '32.5': [0, 5], '34.25': [6, 2], '35.5': [1, 4], '36': [0, 4], '36.5': [1, 4], '40.5': [2, 5], '42.25': [0, 3], '43.5': [6, 2], '44': [0, 5], '44.5': [1, 5], '48.5': [5, 1], '50.25': [0, 3], '51.5': [6, 2], '52': [0, 5], '52.5': [1, 5], '56.5': [3, 6], '57': [1, 6], '60': [2, 6], '61.5': [5, 1], '62.25': [6, 3], '62.5': [5, 2], '64': [6, 1], '65': [2, 5], '66.25': [2, 6], '67.5': [2, 5], '68': [1, 6], '70.25': [6, 2], '73': [0, 5], '74': [6, 0], '75.5': [1, 5], '80.75': [5, 1], '83.5': [1, 5], '85.5': [4, 2], '88.5': [1, 6], '90.25': [6, 2], '91.5': [1, 5], '92': [0, 5], '92.5': [1, 5], '96': [0, 5], '96.5': [1, 5], '97.5': [5, 2], '98.25': [6, 2], '99.5': [1, 4], '100': [6, 0], '100.5': [5, 0], '104.5': [4, 1], '106.25': [0, 3], '107.5': [6, 2], '108': [0, 4], '108.5': [1, 4], '112': [0, 5], '112.5': [1, 5], '114.25': [6, 2]

In [72]:
key_shape = {
    "Single": " (===========) ", 
    "Flick": " (^^^^^^^^^^^) ", 
    "Hold_start": " /===========\\ ", 
    "Hold_end": " \\===========/ ", 
    "Hold_body": "  +         +  ", 
    "Hold_node": "  +   ---   +  ", 
    "Left": [
        " <===========0 ", 
        " <<<=======================0   ", 
        " <<<<<===================================0     "
    ], 
    "Right": [
        " 0===========> ", 
        "   0=======================>>> ", 
        "     0===================================>>>>> "
    ]
}
chart_pos = {
    "left": [1, 17, 33, 49, 65, 81, 97], 
    "right": [15, 31, 47, 63, 79, 95, 111],
    "middel": [8, 24, 40, 56, 72, 88, 104] 
}

In [101]:
bpm = 145
length = 288 + 1
freq = 60 / bpm
expand = 1 / freq
if expand < 1.41: bps = 1
elif expand < 2.82: bps = 2
else: bps = 4
lpb = int(64 / bps)
chart_line = lpb * length
blank = ("|" + " " * 15)*7 + "|"
chart_text = [blank] * chart_line

In [102]:
for single in chart_keys["Single"]:
    line = int((float(single["beat"]) * lpb))
    lane = single["lane"]
    node = chart_pos["left"][lane], chart_pos["right"][lane]
    line_chart = chart_text[line]
    line_chart = line_chart[:node[0]] + key_shape["Single"] + line_chart[node[1]+1:]
    chart_text[line] = line_chart

In [103]:
for single in chart_keys["Flick"]:
    line = int((float(single["beat"]) * lpb))
    lane = single["lane"]
    node = chart_pos["left"][lane], chart_pos["right"][lane]
    line_chart = chart_text[line]
    line_chart = line_chart[:node[0]] + key_shape["Flick"] + line_chart[node[1]+1:]
    chart_text[line] = line_chart

In [104]:
for beat in links.keys():
    line = int((float(beat) * lpb))
    if links[beat][0] < links[beat][1]:
        node = chart_pos["middel"][links[beat][0]], chart_pos["middel"][links[beat][1]]
    else: 
        node = chart_pos["middel"][links[beat][1]], chart_pos["middel"][links[beat][0]]
    line_chart = chart_text[line]
    new_line_chart = ""
    for i in range(len(line_chart)):
        if line_chart[i] in [" ", "|"] and (i >= node[0] and i <= node[1]):
            new_line_chart += "-"
        else:
            new_line_chart += line_chart[i]
    chart_text[line] = new_line_chart

In [105]:
time_step = [0] * len(chart_text)
for stamp in time_stamps:
    index = int((float(stamp["beat"]) * lpb))
    step = 60 / float(stamp["bpm"]) / lpb
    for i in range(index, len(chart_text)):
        time_step[i] = step

In [106]:
for i in range(len(chart_text)):
    print(chart_text[-i-1])

|               |               |               |               |               |               |               |
|               |               |               |               |               |               |               |
|               |               |               |               |               |               |               |
|               |               |               |               |               |               |               |
|               |               |               |               |               |               |               |
|               |               |               |               |               |               |               |
|               |               |               |               |               |               |               |
|               |               |               |               |               |               |               |
|               |               |               |               |               |       